In [35]:
#Import lIBRARIES
import pandas as pd
import numpy as np
import os as os

In [36]:
# example making new folder with os

os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['basics_clean.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title.akas.csv.gz',
 'title.ratings.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

# Basics Data Load & Clean

In [45]:
##Load Basics set and assign to DF

basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [46]:
#Assign local file to DF
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [47]:
#Basics preprocessing

#Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True) 
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [49]:
#Eliminate movies that are null for runtimeMinutes
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])

In [50]:
basics['genres'].isna().sum()

0

In [51]:
#keep only titleType==Movie
movie_filter = basics['titleType']=='movie'

basics = basics[movie_filter]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1172,tt0001184,movie,Don Juan de Serrallonga,Don Juan de Serrallonga,0,1910,NaN,58,"Adventure,Drama"
1273,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"


In [52]:
#basics['runtimeMinutes'].dropna(inplace=True)
#basics.head()

In [53]:
#keep startYear 2000-2022
start_filt = basics['startYear'] >= '2000'
start_filt.head()

570     False
587     False
672     False
1172    False
1273    False
Name: startYear, dtype: bool

In [54]:
end_filt = basics['startYear']<='2022'
end_filt.head()

570     True
587     True
672     True
1172    True
1273    True
Name: startYear, dtype: bool

In [55]:
basics = basics.loc[start_filt & end_filt, :]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77968,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"


In [56]:
#Eliminate movies that include  "Documentary" in genre (see tip below)
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77968,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86806,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [57]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140077 entries, 34805 to 9001543
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          140077 non-null  object
 1   titleType       140077 non-null  object
 2   primaryTitle    140077 non-null  object
 3   originalTitle   140077 non-null  object
 4   isAdult         140077 non-null  object
 5   startYear       140077 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  140077 non-null  object
 8   genres          140077 non-null  object
dtypes: object(9)
memory usage: 10.7+ MB


# AKAs Data Load & Clean

In [58]:
##Load AKAs data set and save locally 
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

akas.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [59]:
#Assign local file to DF
akas = pd.read_csv("Data/title.akas.csv.gz", low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [60]:
#AKA Preprocessing
akas.replace({'\\N':np.nan}, inplace=True) 

In [61]:
#AKA - keep only US entries. I keep breaking the main df, I'm creating a dummy
#df that I will move back once done.
dfa= akas
dfa.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [62]:
#Apply filter to retain only movies in US region
region_filt = dfa['region']=='US'
region_filt.head()

0    False
1    False
2    False
3    False
4    False
Name: region, dtype: bool

In [63]:
#Check that filtering worked...it did.  ONLY US region films are showing.
dfa[region_filt].head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [64]:
dfa = dfa.loc[region_filt, :]
dfa.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [65]:
dfa['region'].value_counts()

US    1328708
Name: region, dtype: int64

In [66]:
#Transfer dummy set back to actual df
akas = dfa
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [67]:
#Save new cleansed df to local csv file from earlier 
akas.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [68]:
#Display df info as requested
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1328708 entries, 5 to 32328026
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1328708 non-null  object
 1   ordering         1328708 non-null  int64 
 2   title            1328708 non-null  object
 3   region           1328708 non-null  object
 4   language         3581 non-null     object
 5   types            1026140 non-null  object
 6   attributes       44187 non-null    object
 7   isOriginalTitle  1327333 non-null  object
dtypes: int64(1), object(7)
memory usage: 91.2+ MB


# Ratings Data Load & Clean

In [69]:
##Load Ratings data set and assign to DF
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

ratings.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

In [70]:
#Assign local file to DF
ratings = pd.read_csv("Data/title.ratings.csv.gz", low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1888
1,tt0000002,5.9,251
2,tt0000003,6.5,1679
3,tt0000004,5.8,164
4,tt0000005,6.2,2498


In [71]:
#RATINGS Preprocessing
ratings.replace({'\\N':np.nan}) 

,tconst,averageRating,numVotes
0,tt0000001,5.7,1888
1,tt0000002,5.9,251
2,tt0000003,6.5,1679
3,tt0000004,5.8,164
4,tt0000005,6.2,2498
...,...,...,...
1254181,tt9916690,6.5,6
1254182,tt9916720,5.3,223
1254183,tt9916730,8.4,6
1254184,tt9916766,6.7,20


In [72]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254186 entries, 0 to 1254185
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1254186 non-null  object 
 1   averageRating  1254186 non-null  float64
 2   numVotes       1254186 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.7+ MB


In [73]:
ratings.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

# Filtering Dataframes Upon One Another

In [74]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34805       True
61119       True
67672       True
77968      False
86806       True
           ...  
9001366     True
9001375     True
9001414    False
9001459     True
9001543    False
Name: tconst, Length: 140077, dtype: bool

In [75]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61119,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67672,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86806,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
91077,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"
...,...,...,...,...,...,...,...,...,...
9000830,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9001226,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9001366,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9001375,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,90,"Action,Adventure,Thriller"


In [76]:
basics.to_csv("Data/basics_clean.csv.gz",compression='gzip',index=False)